<a href="https://colab.research.google.com/github/JengkolRebus/kalibrasi-arah-kiblat/blob/main/kalibrasiArahKiblat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# jengkolrebus
# 31 January 2021
# Yogyakarta
!pip install skyfield

     |████████████████████████████████| 389kB 5.4MB/s 
     |████████████████████████████████| 40kB 3.3MB/s 
     |████████████████████████████████| 266kB 7.5MB/s 
  Created wheel for skyfield: filename=skyfield-1.36-cp36-none-any.whl size=428757 sha256=2f7a803f4481b752741cfe571787ca554ba2830ea02c6439529c8b8e8d3d78cf
  Stored in directory: /root/.cache/pip/wheels/11/98/cd/cdfe900b77701df1db4c1ff3424af3eb8a03b12966def2eacd
  Created wheel for jplephem: filename=jplephem-2.15-cp36-none-any.whl size=46018 sha256=dc31499179e1dddafdb80401d78b4f7039f90f1134202eaa4db09fc7d3ff38f4
  Stored in directory: /root/.cache/pip/wheels/01/71/9c/beaa33beb607ef67b62765324a10cd3584e30fbeef8856b7a8
Successfully built skyfield jplephem


In [ ]:
from skyfield.api import load, Topos
from datetime import datetime, timedelta
from pytz import timezone
import calendar

from ipywidgets import widgets, interact, interactive
from IPython.display import display, HTML

import numpy as np


jkt = timezone('Asia/Jakarta')
utc = timezone('UTC')
ts = load.timescale()
e = load('de421.bsp')

location = ['7.83305556 S','110.38305556 E']
lintangKabah = np.radians(21+(25/60)+(21.07/3600))
bujurKabah = np.radians(39+(49/60)+(34.37/3600))
lintangTempat = np.radians(float(location[0].split(' ')[0])*-1)
bujurTempat = np.radians(float(location[1].split(' ')[0]))

def arahKiblat(lintangKabah, bujurKabah, lintangTempat, bujurTempat):
    a = np.sin(bujurTempat-bujurKabah)
    b = np.cos(lintangTempat)*np.tan(lintangKabah)
    c = np.sin(lintangTempat)*np.cos(bujurTempat - bujurKabah)
    K = np.degrees(np.arctan(a/(b-c)))
    az = 270+(90-K)
    K = toDMS(K)
    az = toDMS(az)
    return K, az

def toDMS(degree):
    d = int(degree)
    m = int((degree-d)*60)
    s = (degree - d - m / 60) * 3600
    degree_sign = u'\N{DEGREE SIGN}'
    return '{}{} {}\' {}\"'.format(d, degree_sign, m, round(s, 2))

def result(latitude, longitude, time):
    t = utc.localize(time)
    t = ts.from_datetime(t)
    loc = e['earth'] + Topos(latitude, longitude)
    astrometric = loc.at(t).observe(e['sun'])
    alt, az, d = astrometric.apparent().altaz()
    return t.astimezone(jkt), az

In [89]:


current_day = datetime.now()
time = widgets.DatePicker(value=current_day, description='Waktu:',
                          disabled=False)
def currTime():
    current_daytime = datetime.now()
    time.value = current_daytime

lat = widgets.Text(value=location[0], description='Latitude:')
longi = widgets.Text(value=location[1], description='Longitude:')

display(lat)
display(longi)

button = widgets.Button(description="Find Sun")
output = widgets.Output()
display(button, output)

def on_button_clicked(b):
    currTime()
    t, az = result(lat.value, longi.value, time.value)
    print('Waktu:', t)
    print('Azimuth Matahari:', az)
    print()
    arah_kiblat, az_kiblat = arahKiblat(lintangKabah, bujurKabah, lintangTempat, bujurTempat)
    print('Arah Kiblat:', arah_kiblat, 'Dari Utara ke Barat')
    print('Azimuth Kiblat:', az_kiblat)

button.on_click(on_button_clicked)

Text(value='7.83305556 S', description='Latitude:')

Text(value='110.38305556 E', description='Longitude:')

Button(description='Find Sun', style=ButtonStyle())

Output()

Waktu: 2021-01-31 17:08:40.122138+07:00
Azimuth Matahari: 253deg 58' 03.1"

Arah Kiblat: 65° 16' 59.4" Dari Utara ke Barat
Azimuth Kiblat: 294° 43' 0.6"
